In [2]:
from keras.models import load_model
import cv2
import numpy as np

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Rescaling
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [4]:
# from splitfolders import ratio

# input_folder = 'ds_c'
# ratio(input_folder, 'ds_splited2', ratio=(0.6, 0.25, 0.15), seed=13)
# print('Data was splitted')

In [5]:
batch_size = 64
img_width = 45
img_height = 45
image_size = (img_width, img_height)
input_shape = (img_width, img_height, 1)
epochs = 100

In [6]:
train_ds = image_dataset_from_directory(
    'dataset_splited/train',
    image_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
)

Found 19613 files belonging to 70 classes.


In [7]:
val_ds = image_dataset_from_directory(
    'dataset_splited/val',
    image_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale'
)

Found 8159 files belonging to 70 classes.


In [8]:
test_ds = image_dataset_from_directory(
    'dataset_splited/test',
    image_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale'
)

Found 4961 files belonging to 70 classes.


In [9]:
class_names = train_ds.class_names
nb_classes = len(class_names)

In [10]:
# print(class_names)
# print(nb_classes)


# for image_batch, label_batch in train_ds:
#     print(image_batch.shape)
#     print(label_batch)
#     break


# plt.figure(figsize=(5, 5))
# for images, labels in train_ds.take(1):
#   lbl=labels.numpy()
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("int"), cmap='gray')
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

In [12]:
model = Sequential([
    Rescaling(1./255),
    Conv2D(8, 3, padding='same', activation='relu', input_shape=input_shape),
    MaxPooling2D(),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dense(nb_classes, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    metrics=['accuracy']
)
callback = [EarlyStopping(monitor = 'val_loss', 
                         patience=2, 
                         restore_best_weights=True),
]


In [13]:
history = model.fit(train_ds,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[callback]
)

Epoch 1/100
307/307 [==============================] - 21s 42ms/step - loss: 3.5447 - accuracy: 0.1702 - val_loss: 1.8179 - val_accuracy: 0.5550
Epoch 2/100
307/307 [==============================] - 11s 36ms/step - loss: 1.6796 - accuracy: 0.5573 - val_loss: 1.1815 - val_accuracy: 0.6905
Epoch 3/100
144/307 [=============>................] - ETA: 4s - loss: 1.2919 - accuracy: 0.6485

KeyboardInterrupt: 

In [ ]:
model.save('models/hmer_model_2.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history['loss']))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
# plt.show()
plt.savefig('model_plt.png')


In [ ]:
print('Training accuracy: ', '{:.3f}'.format(acc[-1]))
print('Validation accuracy: ', '{:.3f}'.format(val_acc[-1]))
print('Training loss: ', '{:.3f}'.format(loss[-1]))
print('Validation loss: ', '{:.3f}'.format(val_loss[-1]))

In [ ]:
results = model.evaluate(test_ds, verbose=1)
print('Test accuracy: ', '{:.3f}'.format(results[1]))
print('Test loss: ', '{:.3f}'.format(results[0]))

In [ ]:
y_pred = []
y_true = [] 

for image_batch, label_batch in test_ds: 
   y_true.append(label_batch)
   preds = model.predict(image_batch)
   y_pred.append(np.argmax(preds, axis = 1))

correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('confusion_matrix.png')

confusion_mtx = confusion_matrix(correct_labels, predicted_labels) 

plt.figure(figsize=(18, 18))
plot_confusion_matrix(confusion_mtx, classes = range(nb_classes)) 